## Final Project Submission

Please fill out:
* **Student name**: Julius Kinyua
* **Student pace**: Full time
* **Scheduled project review date/time**: 22/03/2024
* **Instructor name**: Asha Deen
* **Blog post URL**: https://lyonec.com/


# Section 1: Business Understanding

Microsoft has started a new movie studio and intends to understand the best-performing movies. As a data scientist, my role is to perform in-depth analysis on existing movie data from various sources.

The sources of data being analyzed in this case include:
- IMDb data
- Movies data
- Movie reviews
- Movie budget
- Among others

After a thorough analysis of the existing data, this notebook includes recommendations on the best movies and movie genres that Microsoft Studio should focus on investing in.

# Section 2: Data Understanding

In [2]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

sns.set_theme(style="darkgrid")

In [8]:
# Unzip the zipped im.db using zipfile module
path = "zippedData/im.db.zip"

with ZipFile(path, 'r') as zipf:
    zipf.extractall('data')


In [10]:
# Create a connection to the Database:
conn = sqlite3.connect('data/im.db')

query = """
SELECT name
FROM sqlite_master
WHERE type = 'table'
""";

pd.read_sql(query, conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


The line of code above use the query line above to check the table names in the im.db database contained in the
im.db databse that has been unzipped above.

### Tables To Use For Analysis

From the sqlite query results obtained above, and with regard to our main goals for data analysis, we can draw that the most important tables to use include:
- movie_basics
- movie_ratings

Now, let's go ahead and query the tables to see the first five and last rows of each table.

In [11]:
movie_basics_query = """
SELECT *
FROM movie_basics
"""

pd.read_sql(movie_basics_query, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [13]:
movie_ratings_query = """
SELECT *
FROM movie_ratings
"""

pd.read_sql(movie_ratings_query, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


From the two DataFrames obtained above, we can use the movie_id to check the movie basics and movie rating of individual films
using a one to one relationship

### Structure of the tables

Let's check the structure and descriptive statistics of each table so that we can understand the data type of each of the columns.

In [15]:
# This line ehecks the number of columns as well the datatype contained in movie_basics table

pd.read_sql("SELECT * FROM movie_basics", conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [17]:
# Let's have a closer look at some descriptive statistics on movie_basics table

pd.read_sql("SELECT * FROM movie_basics", conn).describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [16]:
# This line ehecks the number of columns as well the datatype contained in movie_ratings table

pd.read_sql("SELECT * FROM movie_ratings", conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [18]:
# Let's have a closer look at some descriptive statistics on movie_ratings table

pd.read_sql("SELECT * FROM movie_ratings", conn).describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


Next, we go ahead to check the size of each Dataframe

In [ ]:
movie_basics_size = pd.read_sql("SELECT * FROM movie_basics;", conn).shape()
movie_ratings_size = pd.read_sql("SELECT * FROM movie_ratings;", conn).shape()

# Section 3: Data Cleaning

# Section 4: Data Analysis

# Section 5: Conclusion

# Section 6: Recommendations

# Section 7: Next Steps